# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/qTEA1/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298352340963                   -0.85    5.1         
  2   -8.300194173600       -2.73       -1.26    1.1    100ms
  3   -8.300431488042       -3.62       -1.89    3.0    130ms
  4   -8.300461299424       -4.53       -2.75    2.0    111ms
  5   -8.300463781323       -5.61       -3.03    4.2    165ms
  6   -8.300464224788       -6.35       -3.23    1.5    149ms
  7   -8.300464431422       -6.68       -3.38    7.6    210ms
  8   -8.300464547040       -6.94       -3.53    2.4    138ms
  9   -8.300464615247       -7.17       -3.72    1.5    124ms
 10   -8.300464634563       -7.71       -3.90    2.5    146ms
 11   -8.300464643432       -8.05       -4.24    3.0    190ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67446446634                   -0.70    5.6         
  2   -16.67868068707       -2.38       -1.14    1.9    281ms
  3   -16.67920606434       -3.28       -1.87    3.2    319ms
  4   -16.67927890670       -4.14       -2.76    3.6    346ms
  5   -16.67928594687       -5.15       -3.19    4.1    379ms
  6   -16.67928619072       -6.61       -3.48    2.1    291ms
  7   -16.67928621684       -7.58       -3.93    1.8    256ms
  8   -16.67928622136       -8.34       -4.55    3.0    319ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32601028520                   -0.56    7.9         
  2   -33.33411990989       -2.09       -1.00    1.6    1.01s
  3   -33.33599524443       -2.73       -1.71    4.5    1.32s
  4   -33.33618088559       -3.73       -2.64    5.2    1.23s
  5   -33.33692751255       -3.13       -2.71    8.8    1.65s
  6   -33.33693815456       -4.97       -2.97    3.4    1.06s
  7   -33.33694374243       -5.25       -4.00    2.5    1.13s
  8   -33.33694378016       -7.42       -4.35    5.1    1.48s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298291344587                   -0.85    5.1         
  2   -8.300234314972       -2.71       -1.59    1.0    102ms
  3   -8.300441771731       -3.68       -2.64    3.0    125ms
  4   -8.300425159409   +   -4.78       -2.46    8.0    222ms
  5   -8.300464344090       -4.41       -3.48    1.0   92.0ms
  6   -8.300464591828       -6.61       -3.87    3.0    143ms
  7   -8.300464641973       -7.30       -4.50    1.4    111ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32564199401                   -0.56    7.4         
  2   -33.32872202751       -2.51       -1.27    1.2    834ms
  3   -24.18219887703   +    0.96       -0.59    5.8    1.53s
  4   -33.26109638551        0.96       -1.62    5.0    1.49s
  5   -33.18586523643   +   -1.12       -1.45    4.0    1.26s
  6   -33.16261096860   +   -1.63       -1.44    4.4    1.23s
  7   -33.33054803984       -0.77       -1.97    3.2    1.09s
  8   -33.33397679101       -2.46       -2.29    2.6    1.00s
  9   -33.33528440363       -2.88       -2.41    2.6    1.02s
 10   -33.33519213782   +   -4.03       -2.42    1.4    852ms
 11   -33.33653708075       -2.87       -2.70    1.8    907ms
 12   -33.33692066268       -3.42      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.